In [1]:
import dotenv
dotenv.load_dotenv("../backend/.env")

True

In [2]:
import os
import pandas as pd
from langchain.vectorstores import Chroma
from unixcoder import UnixcoderEmbeddings
from IPython.core.display import display, HTML
from langchain.embeddings.openai import OpenAIEmbeddings
from evaluation import run_pair_comparison_multiple

/home/JanVincent.Hoffbauer/codemate/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_802512/440255638.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
CONTEXT_LENGTH = 10

In [12]:
db_unix = Chroma(persist_directory="./unixcoder_embeddings", embedding_function=UnixcoderEmbeddings())
db_openai = Chroma(persist_directory="./openai_embeddings", embedding_function=OpenAIEmbeddings())

results = []

for filename in os.listdir("test_data"):
    with open(os.path.join("test_data", filename)) as f:
        full_text = f.read()

    text_lines = full_text.split("\n")
    cursor_index = [i for i, line in enumerate(text_lines) if "[CURSOR]" in line][0]
    text_lines = text_lines[max(0, cursor_index - CONTEXT_LENGTH):cursor_index + CONTEXT_LENGTH + 1]
    text = "\n".join(text_lines).replace("[CURSOR]", "")

    snippets_unix = db_unix.similarity_search(text, k=3)
    snippets_openai = db_openai.similarity_search(text, k=3)

    snippets_unix = [s.page_content[:2000] for s in snippets_unix]
    snippets_openai = [s.page_content[:2000] for s in snippets_openai]

    # try: 
    #     evaluation_result = run_pair_comparison_multiple(text, snippet_unix, snippet_openai)
    #     evaluation_result = "UniXcoder" if evaluation_result == 0 else "OpenAI"
    # except:
    #     evaluation_result = "No Decision"

    result = {}
    result["Filename"] = filename
    result["Example"] = full_text
    for i in range(3): 
        result[f"UniXcoder #{i}"] = snippets_unix[i]
    for i in range(3): 
        result[f"OpenAI #{i}"] = snippets_openai[i]
    # result["GPT Evaluation"] = evaluation_result
    results.append(result)

In [13]:
len(results)

11

In [14]:
df = pd.DataFrame.from_dict(results, orient="columns")

In [15]:
def format_code(code):
    # Escape
    code = code.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
    # Add line breaks
    code = code.replace('\n', '<br>')
    # Make code block
    code = f'<pre><code>{code}</code></pre>'
    # Make leftbound 
    code = f'<div style="text-align: left; width: 300px; word-wrap: break-word; white-space: normal; overflow-x: scroll">{code}</div>'
    return code

html = df.to_html(escape=False, formatters = {'Example': format_code, 
                                              'UniXcoder #0': format_code, 'UniXcoder #1': format_code, 'UniXcoder #2': format_code,
                                              'OpenAI #0': format_code, 'OpenAI #1': format_code, 'OpenAI #2': format_code,}, index=True)
with open("test_results_k3.html", "w") as f:
    f.write(html)
    
display(HTML(html))